## setup

In [1]:
import Interface as I

/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:13: UserWarning: environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. Falling back to default configuration.
  warnings.warn('environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. ' +
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:33: UserWarning: Using file based locking.Please be careful on nfs mounts as file based locking has issues in this case.
  warnings.warn('Using file based locking.'


trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'spock', 'port': 8885, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'localhost', 'port': 6379, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'file'}
Current version: heads/vcbm-2023
Current pid: 304255


/scratch/visualn/bzfharth/in-silico-install-dir/anaconda_isf3/lib/python3.8/site-packages/pandas_msgpack-0.1.4+14.gfcb0471-py3.8-linux-x86_64.egg/pandas_msgpack/packers.py:78: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Timestamp, Period, Series, DataFrame,  # noqa
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/visualize/cell_morphology_visualizer.py:29: ImportWarning: Dash could not be imported. Interactive visualisations will not work.
  warnings.warn(


Loading mechanisms:
setting up local multiprocessing framework ... done


Loaded modules with __version__ attribute are:
IPython: 7.19.0, Interface: heads/vcbm-2023, PIL: 8.2.0, _csv: 1.0, _ctypes: 1.1.0, _curses: b'2.2', _decimal: 1.70, argparse: 1.1, backcall: 0.2.0, blosc: 1.10.2, bluepyopt: 1.9.126, bottleneck: 1.3.2, cffi: 1.14.3, click: 7.1.2, cloudpickle: 1.6.0, colorama: 0.4.4, csv: 1.0, ctypes: 1.1.0, cycler: 0.10.0, cytoolz: 0.11.0, dask: 2.30.0, dateutil: 2.8.1, deap: 1.3, decimal: 1.70, decorator: 4.4.2, distributed: 2.30.1, distutils: 3.8.5, fsspec: 0.8.3, future: 0.18.2, ipykernel: 5.3.4, ipython_genutils: 0.2.0, ipywidgets: 7.5.1, jedi: 0.17.1, jinja2: 2.11.2, joblib: 0.17.0, json: 2.0.9, jupyter_client: 6.1.7, jupyter_core: 4.6.3, kiwisolver: 1.3.0, logging: 0.5.1.2, markupsafe: 1.1.1, matplotlib: 3.3.2, mkl: 2.3.0, neuron: 7.8.2, numexpr: 2.7.1, numpy: 1.19.2, pandas: 1.1.3, pandas_msgpack: 0.1.4+14.gfcb0471.dirty, parameters: 0.2.1, parso: 0.7.0, past: 0.18.2, pexpect

In [2]:
mdb = I.ModelDataBase('/scratch/visualn/bzfharth/simulation_data_2023-06-21')
print(mdb.keys())

['89', '90', 'WR69', '88', '93', '84', '87', 'WR64', '85', '86', '91', 'WR71', 'examplary_models', 'df_all', 'run_gillis_protocol_v1', 'all_models_py3', 'run_gillis_protocol_v1__all_models_genetic_py3', 'all_models_genetic_py3', 'all_models_genetic_py2_1', 'all_models_genetic_py2_2']


In [3]:
import os
import shutil

def makeCleanDir(dirname):
    if(os.path.exists(dirname)):
        shutil.rmtree(dirname, ignore_errors=False, onerror=None)
    os.mkdir(dirname)

selected_dataset = "all_models_py3"
postfix = "_2023-06-21"
csv_outfolder = os.path.join('/scratch/visualn/bzfharth/simulation_data_postprocessed', selected_dataset+postfix)

makeCleanDir(csv_outfolder)

In [4]:
# init
import pickle
import biophysics_fitting.hay_evaluation
import project_specific_ipynb_code.biophysical_models
from project_specific_ipynb_code.biophysical_models import get_model_ddf_from_encapsulating_mdb, objectives_BAC
from project_specific_ipynb_code.biophysical_models import get_ddf_selected
from project_specific_ipynb_code.hot_zone import get_cell_object_from_hoc
from project_specific_ipynb_code.biophysical_models import min_max_plot, normalize_params
from project_specific_ipynb_code.biophysical_models import CurrentAnalysis
from project_specific_ipynb_code.biophysical_models import modify_simulator_to_record_apical_dendrite_conductances

In [5]:
# init
objectives_BAC = project_specific_ipynb_code.biophysical_models.objectives_BAC
objectives_step = project_specific_ipynb_code.biophysical_models.objectives_step
objectives_2BAC = ['1BAC_APheight', '1BAC_ISI', '1BAC_ahpdepth', '1BAC_caSpike_height', '1BAC_caSpike_width', '1BAC_spikecount', '2BAC_APheight', '2BAC_ISI', '2BAC_ahpdepth', '2BAC_caSpike_height', '2BAC_caSpike_width', '2BAC_spikecount', 'bAP_APheight', 'bAP_APwidth', 'bAP_att2', 'bAP_att3', 'bAP_spikecount']

In [6]:
# they are the same for all optimizations with the exception of CaDynamics_E2 (py2 simulations) 
# and CaDynamics_E2_v2 (py3 simulations)

params_py2 = ['ephys.CaDynamics_E2.apic.decay',
       'ephys.CaDynamics_E2.apic.gamma', 'ephys.CaDynamics_E2.axon.decay',
       'ephys.CaDynamics_E2.axon.gamma', 'ephys.CaDynamics_E2.soma.decay',
       'ephys.CaDynamics_E2.soma.gamma', 'ephys.Ca_HVA.apic.gCa_HVAbar',
       'ephys.Ca_HVA.axon.gCa_HVAbar', 'ephys.Ca_HVA.soma.gCa_HVAbar',
       'ephys.Ca_LVAst.apic.gCa_LVAstbar', 'ephys.Ca_LVAst.axon.gCa_LVAstbar',
       'ephys.Ca_LVAst.soma.gCa_LVAstbar', 'ephys.Im.apic.gImbar',
       'ephys.K_Pst.axon.gK_Pstbar', 'ephys.K_Pst.soma.gK_Pstbar',
       'ephys.K_Tst.axon.gK_Tstbar', 'ephys.K_Tst.soma.gK_Tstbar',
       'ephys.NaTa_t.apic.gNaTa_tbar', 'ephys.NaTa_t.axon.gNaTa_tbar',
       'ephys.NaTa_t.soma.gNaTa_tbar', 'ephys.Nap_Et2.axon.gNap_Et2bar',
       'ephys.Nap_Et2.soma.gNap_Et2bar', 'ephys.SK_E2.apic.gSK_E2bar',
       'ephys.SK_E2.axon.gSK_E2bar', 'ephys.SK_E2.soma.gSK_E2bar',
       'ephys.SKv3_1.apic.gSKv3_1bar', 'ephys.SKv3_1.apic.offset',
       'ephys.SKv3_1.apic.slope', 'ephys.SKv3_1.axon.gSKv3_1bar',
       'ephys.SKv3_1.soma.gSKv3_1bar', 'ephys.none.apic.g_pas',
       'ephys.none.axon.g_pas', 'ephys.none.dend.g_pas',
       'ephys.none.soma.g_pas', 'scale_apical.scale']

params_py3 = [p.replace('ephys.CaDynamics_E2', 'ephys.CaDynamics_E2_v2') for p in params_py2]

In [7]:
df = mdb[selected_dataset]
print(df.shape)

(1408955, 98)


In [8]:
feature_names = set()
blocked = set(['scale_apical.scale', 'model_id', 'type_',
       'morphology', 'lv', 'seed', 'max_', 'gen',
       'hot_zone.outsidescale_sections', 'BAC.hay_measure.recSite',
       'BAC.stim.dist', 'bAP.hay_measure.recSite1', 'bAP.hay_measure.recSite2',
       'hot_zone.min_', 'hot_zone.max_', 'morphology.filename', 'sort_column'])
for col_name in df.columns:
    if("ephys" not in col_name):
        feature_names.add(col_name)

feature_names = sorted(feature_names-blocked)
feature_names

['1BAC_APheight',
 '1BAC_ISI',
 '1BAC_ahpdepth',
 '1BAC_caSpike_height',
 '1BAC_caSpike_width',
 '1BAC_spikecount',
 '2BAC_APheight',
 '2BAC_ISI',
 '2BAC_ahpdepth',
 '2BAC_caSpike_height',
 '2BAC_caSpike_width',
 '2BAC_spikecount',
 'AI1',
 'AI2',
 'AI3',
 'APh1',
 'APh2',
 'APh3',
 'APw1',
 'APw2',
 'APw3',
 'DI1',
 'DI2',
 'DI3',
 'ISIcv1',
 'ISIcv2',
 'ISIcv3',
 'TTFS1',
 'TTFS2',
 'TTFS3',
 'bAP_APheight',
 'bAP_APwidth',
 'bAP_att2',
 'bAP_att3',
 'bAP_spikecount',
 'fAHPd1',
 'fAHPd2',
 'fAHPd3',
 'mf1',
 'mf2',
 'mf3',
 'sAHPd1',
 'sAHPd2',
 'sAHPd3',
 'sAHPt1',
 'sAHPt2',
 'sAHPt3']

In [9]:
parameters = df[params_py3]
parameters.to_csv(os.path.join(csv_outfolder, "parameters.csv"))
parameters

,ephys.CaDynamics_E2_v2.apic.decay,ephys.CaDynamics_E2_v2.apic.gamma,ephys.CaDynamics_E2_v2.axon.decay,ephys.CaDynamics_E2_v2.axon.gamma,ephys.CaDynamics_E2_v2.soma.decay,ephys.CaDynamics_E2_v2.soma.gamma,ephys.Ca_HVA.apic.gCa_HVAbar,ephys.Ca_HVA.axon.gCa_HVAbar,ephys.Ca_HVA.soma.gCa_HVAbar,ephys.Ca_LVAst.apic.gCa_LVAstbar,...,ephys.SKv3_1.apic.gSKv3_1bar,ephys.SKv3_1.apic.offset,ephys.SKv3_1.apic.slope,ephys.SKv3_1.axon.gSKv3_1bar,ephys.SKv3_1.soma.gSKv3_1bar,ephys.none.apic.g_pas,ephys.none.axon.g_pas,ephys.none.dend.g_pas,ephys.none.soma.g_pas,scale_apical.scale
0,59.047725,0.001129,743.738608,0.001219,159.291539,0.013790,0.003765,0.000280,0.000048,0.194205,...,0.006276,0.029057,-1.588595,0.270214,1.231999,0.000059,0.000029,0.000030,0.000044,1.557644
1,58.543237,0.001129,743.738608,0.001219,159.291539,0.013790,0.003765,0.000280,0.000048,0.194205,...,0.006277,0.029057,-1.588595,0.270214,1.231999,0.000059,0.000038,0.000032,0.000044,1.557644
2,58.484004,0.001129,743.738608,0.001219,159.291539,0.013790,0.003765,0.000280,0.000048,0.194302,...,0.005889,0.029057,-1.588595,0.270214,1.231999,0.000059,0.000030,0.000030,0.000044,1.557644
3,58.484004,0.001129,743.738608,0.001219,159.291539,0.013790,0.003765,0.000280,0.000048,0.194289,...,0.005889,0.029057,-1.588595,0.270214,1.231999,0.000059,0.000030,0.000030,0.000044,1.557644
4,58.483893,0.001129,741.946549,0.001219,159.291539,0.013790,0.003765,0.000280,0.000048,0.195171,...,0.005889,0.029057,-1.588595,0.270250,1.231999,0.000059,0.000030,0.000030,0.000044,1.558713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408950,56.784256,0.000695,556.167800,0.000640,145.723416,0.001800,0.002873,0.000505,0.000424,0.099105,...,0.004512,0.175263,-2.417505,0.853836,0.762143,0.000059,0.000032,0.000042,0.000027,1.348737
1408951,64.305909,0.000681,558.803676,0.000640,144.561063,0.001789,0.002872,0.000513,0.000542,0.099141,...,0.004512,0.175522,-2.382783,0.856131,0.762149,0.000059,0.000033,0.000042,0.000026,1.348715
1408952,64.305909,0.000681,558.803676,0.000640,144.561063,0.001789,0.002872,0.000513,0.000542,0.099141,...,0.004512,0.175522,-2.382783,0.855919,0.762149,0.000059,0.000033,0.000042,0.000026,1.348715
1408953,56.784256,0.000612,835.176942,0.000639,146.287195,0.001801,0.002146,0.000505,0.000428,0.099105,...,0.004512,0.176695,-2.417505,0.854053,0.762143,0.000059,0.000032,0.000037,0.000027,1.348693


In [10]:
features = df[feature_names]
features.to_csv(os.path.join(csv_outfolder, "features.csv"))
features

,1BAC_APheight,1BAC_ISI,1BAC_ahpdepth,1BAC_caSpike_height,1BAC_caSpike_width,1BAC_spikecount,2BAC_APheight,2BAC_ISI,2BAC_ahpdepth,2BAC_caSpike_height,...,fAHPd3,mf1,mf2,mf3,sAHPd1,sAHPd2,sAHPd3,sAHPt1,sAHPt2,sAHPt3
0,0.860674,2.378772,0.313470,0.052148,1.098425,0.0,1.070249,0.441470,0.354247,0.267721,...,0.769300,2.272727,3.571429,4.050041,0.800683,0.126106,0.214718,0.207266,1.174189,3.313219
1,0.850648,2.393448,0.322252,0.045527,1.118110,0.0,0.978449,1.131267,0.412842,0.212028,...,0.772730,1.704545,3.571429,4.050041,0.808328,0.128048,0.213264,0.162391,1.041665,3.318762
2,0.851402,2.525537,0.330870,0.290714,1.472441,0.0,1.026520,1.116590,0.377576,0.040474,...,0.761136,2.272727,3.571429,4.050041,0.799019,0.123177,0.218452,0.246932,1.216245,3.368594
3,0.839281,2.540214,0.331027,0.295573,1.511811,0.0,1.037798,1.131267,0.375546,0.029042,...,0.761885,2.272727,3.571429,4.050041,0.798471,0.123395,0.218107,0.220666,1.265364,3.446314
4,0.835922,2.554890,0.334502,0.311170,1.472441,0.0,0.999727,1.219326,0.385494,0.016952,...,0.757620,2.272727,3.571429,4.050041,0.798060,0.122046,0.219780,0.256056,1.221613,3.389375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408950,0.987783,0.001174,0.977178,0.003039,2.673228,0.0,1.024086,0.732652,0.957709,0.279456,...,1.493211,1.136364,0.892857,1.125011,1.380413,0.706890,0.597028,2.922478,3.041761,3.019558
1408951,1.037066,0.541857,0.956075,0.018490,0.023622,0.0,1.056875,1.833392,0.965843,0.538986,...,1.960513,1.136364,0.892857,1.125011,1.466678,0.839420,0.907419,2.548827,1.387153,2.504104
1408952,1.065936,0.556534,0.957329,0.024500,0.023622,0.0,1.092118,1.848069,0.967224,0.544259,...,1.963364,1.136364,0.892857,1.125011,1.467359,0.839934,0.908718,2.494041,1.369773,2.483160
1408953,1.027666,1.539861,1.045660,1.210270,0.881890,0.0,1.062272,2.596572,1.041619,1.428923,...,1.723226,0.568182,1.785714,2.700027,1.350200,0.678972,0.658732,1.764101,2.960917,2.009686
